In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os, django
from geotext import GeoText

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
from sklearn.manifold import TSNE
from scoping.models import *
from scoping.views import *
from tmv_app.views import *
import scipy.stats as st

now=datetime.datetime.now()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.nmf module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)


In [17]:
q = Query.objects.get(pk=7368)
q = Query.objects.get(pk=8402)
p = q.project
tag = Tag.objects.get(query=q)

In [11]:
docs = tag.doc_set.all()
if not DocOwnership.objects.filter(doc__in=docs, query=q,relevant__gt=0, tag=tag).exists():
    print("stop")

cats = Category.objects.filter(project=p).exclude(name__icontains="hidden>")
df = pd.DataFrame.from_dict(
    DocUserCat.objects.filter(
        category__in=cats,
        doc__in=docs
    ).values(
        'doc__id',
        'category__name',
        'category__level',
        'user__username'
    )
)
df['category__level'] = df['category__level'].astype(str)
df['cat'] = df[['category__level','category__name','user__username']].apply(lambda x: ' - '.join(x), axis=1)
df['val'] = 1

df['cname'] = df[['category__level','category__name']].apply(lambda x: ' - '.join(x), axis=1)

dudf_wide = df[['doc__id','user__username','cname','val']].pivot_table(index=['doc__id','user__username'],columns="cname", values="val").fillna(0)
dudf_wide_r = dudf_wide.reset_index()
dudf_wide_r.columns = ["doc__id",'user__username'] + list(dudf_wide.columns)
dudf_expanded = dudf_wide_r.melt(id_vars=["doc__id","user__username"]).sort_values(['doc__id','variable'])
dudf_expanded['mean'] = dudf_expanded.groupby(['doc__id','variable']).transform('mean')
dudf_expanded = dudf_expanded.rename(columns={
    "doc__id":"doc_id",
    "variable":"category",
    "user__username":"user"
})

wide_df = (df[['doc__id','cat','val']]
       .drop_duplicates()
       .pivot(index='doc__id',columns="cat",values='val')
       .fillna(0)
      )
cs = wide_df.columns
wide_df['doc_id'] = wide_df.index
wide_df = wide_df.reset_index(drop=True)

do_df = pd.DataFrame.from_dict(
    DocOwnership.objects.filter(doc__in=docs, query=q,relevant__gt=0).values(
        'doc__id',
        'user__username',
        'relevant'
    )
)
do_df = (
    do_df.drop_duplicates(['doc__id','user__username'])
    .pivot(index='doc__id',columns="user__username",values="relevant")
)
rs = []
for c in do_df.columns:
    do_df = do_df.rename(columns={c: f'0 - relevant - {c}'})
    rs.append(f'0 - relevant - {c}')
do_df['doc_id'] = do_df.index
do_df = do_df.reset_index(drop=True)

doc_df = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=docs).values(
        'id','title','content'
    )
).rename(columns={"id":"doc_id"})

dudf_expanded = pd.merge(dudf_expanded,doc_df)[["doc_id","title","content","user","category","value","mean"]]

df = pd.merge(doc_df,pd.merge(wide_df,do_df,how="outer"))[["doc_id","title","content"]+list(rs)+list(cs)]

#df = df.loc[:,(df.sum(axis=0) != 0)]

In [20]:
def compare_tag(tag,p):
    docs = tag.doc_set.all()
    if not DocOwnership.objects.filter(doc__in=docs, query=q,relevant__gt=0, tag=tag).exists():
        return pd.DataFrame()

    cats = Category.objects.filter(project=p).exclude(name__icontains="hidden>")
    df = pd.DataFrame.from_dict(
        DocUserCat.objects.filter(
            category__in=cats,
            doc__in=docs
        ).values(
            'doc__id',
            'category__name',
            'category__level',
            'user__username'
        )
    )
    df['category__level'] = df['category__level'].astype(str)
    df['cat'] = df[['category__level','category__name','user__username']].apply(lambda x: ' - '.join(x), axis=1)
    df['val'] = 1
    
    df['cname'] = df[['category__level','category__name']].apply(lambda x: ' - '.join(x), axis=1)
    
    dudf_wide = df[['doc__id','user__username','cname','val']].pivot_table(index=['doc__id','user__username'],columns="cname", values="val").fillna(0)
    dudf_wide_r = dudf_wide.reset_index()
    dudf_wide_r.columns = ["doc__id",'user__username'] + list(dudf_wide.columns)
    dudf_expanded = dudf_wide_r.melt(id_vars=["doc__id","user__username"]).sort_values(['doc__id','variable'])
    dudf_expanded['mean'] = dudf_expanded.groupby(['doc__id','variable']).transform('mean')
    dudf_expanded = dudf_expanded.rename(columns={
        "doc__id":"doc_id",
        "variable":"category",
        "user__username":"user"
    })
    
    wide_df = (df[['doc__id','cat','val']]
           .drop_duplicates()
           .pivot(index='doc__id',columns="cat",values='val')
           .fillna(0)
          )
    cs = wide_df.columns
    wide_df['doc_id'] = wide_df.index
    wide_df = wide_df.reset_index(drop=True)
    
    do_df = pd.DataFrame.from_dict(
        DocOwnership.objects.filter(doc__in=docs, query=q,relevant__gt=0).values(
            'doc__id',
            'user__username',
            'relevant'
        )
    )
    do_df = (
        do_df.drop_duplicates(['doc__id','user__username'])
        .pivot(index='doc__id',columns="user__username",values="relevant")
    )
    rs = []
    for c in do_df.columns:
        do_df = do_df.rename(columns={c: f'0 - relevant - {c}'})
        rs.append(f'0 - relevant - {c}')
    do_df['doc_id'] = do_df.index
    do_df = do_df.reset_index(drop=True)
    
    doc_df = pd.DataFrame.from_dict(
        Doc.objects.filter(pk__in=docs).values(
            'id','title','content'
        )
    ).rename(columns={"id":"doc_id"})
    
    dudf_expanded = pd.merge(dudf_expanded,doc_df)[["doc_id","title","content","user","category","value","mean"]]
    
    df = pd.merge(doc_df,pd.merge(wide_df,do_df,how="outer"))[["doc_id","title","content"]+list(rs)+list(cs)]

    #df = df.loc[:,(df.sum(axis=0) != 0)]
    

    
    df.to_csv(f'../../climate-policy-map/{tag.title}_comparison.csv', index=False)

    writer = pd.ExcelWriter(f'../../climate-policy-map/{tag.title}_comparison.xlsx', engine='xlsxwriter')
    df.to_excel(writer, index=False, sheet_name='comparison')
    dudf_expanded.to_excel(writer, index=False, sheet_name='by_cat_comparison')
    workbook = writer.book
    worksheet = writer.sheets['comparison']
    wrap_format = workbook.add_format({'text_wrap': True})
    worksheet.set_column('D:ZZ', 20)
    worksheet.set_column('C:C', 80, wrap_format)
    worksheet.set_column('B:B', 30, wrap_format)

    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0,col_num,value.split('@')[0],wrap_format)

    for i,row in df.iterrows():
        try:
            worksheet.set_row(i+1,15+15*len(row['content'])//150)
        except:
            worksheet.set_row(i+1,15)
    worksheet.set_row(0, 40)
    worksheet.freeze_panes(1,3)
    
    worksheet = writer.sheets['by_cat_comparison']
    
    worksheet.set_column('D:ZZ', 15)
    worksheet.set_column('C:C', 40)
    worksheet.set_column('B:B', 40, wrap_format)
    worksheet.set_column('D:E', 40, wrap_format)
    
    for i,row in dudf_expanded.iterrows():
        worksheet.set_row(i+1,17+17*len(row['title'])//60)
        
    worksheet.conditional_format(f'G1:G{i+1}', {
        'type': '3_color_scale',
        'min_value': 0,
        'mid_value': 0.5,
        'max_value': 1
    })
    
    writer.save()
    
    return df

df = compare_tag(tag, p)
df.head()

,doc_id,title,content,0 - relevant - maowenxin2014@163.com,0 - relevant - minj,1 - Biblio-/Scientometrics - maowenxin2014@163.com,1 - Biblio-/Scientometrics - minj,1 - Core - maowenxin2014@163.com,1 - Core - minj,2 - Crit Appraisal - maowenxin2014@163.com,2 - Extract - maowenxin2014@163.com,2 - Extract - minj,2 - Screening - maowenxin2014@163.com,2 - Screening - minj,2 - Synthesis - minj
0,3605659,Some reflections on combining meta-analysis an...,Meta-analysis and structural equation modeling...,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3606326,The Struggle for Evidence in Physical and Reha...,Objective The aim of the study was to evaluate...,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3593774,Past and Current Use of Walking Measures for C...,Objectives: To describe walking measurement in...,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,3594279,Progress in Cardiovascular Imaging,None,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3606278,Accredited Endovascular Surgical Neuroradiolog...,Background: Concerns exist that neurosurgery m...,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
for tag in Tag.objects.filter(query=q,pk__gt=5182):
    df = compare_tag(tag, p)


In [ ]:
tag = Tag.objects.get(query__project=p,title__icontains="Oceans")
docs = tag.doc_set.all()

do_df = pd.DataFrame.from_dict(
    DocOwnership.objects.filter(doc__in=docs, query=q,relevant__gt=0).values(
        'doc__id',
        'user__username',
        'relevant'
    )
)
do_df = (
    do_df.drop_duplicates(['doc__id','user__username'])
    .pivot(index='doc__id',columns="user__username",values="relevant")
)

In [ ]:
print(do_df.shape)

do_df.head()

In [12]:
do_df.loc[:,(do_df.sum(axis=0) != 0)].shape

(20, 2)